<h1 class="text-center">NBA Stat</h1>

## Sommaire des fonctionnalités

Pour scrap, ce qui est scrappé pourquoi et comment

## Library

In [2]:
import numpy as np
import matplotlib.pyplot as plt
from sklearn import linear_model
from sklearn.preprocessing import PolynomialFeatures
from sklearn.pipeline import make_pipeline
from sklearn.linear_model import Ridge

## Scraping

### Player scrap

### Team scrap

In [ ]:
from selenium import webdriver
from bs4 import BeautifulSoup
import time

browser = webdriver.Chrome()
browser.get("https://www.nba.com/stats/teams/traditional/?sort=W_PCT&dir=-1")

time.sleep(2)

cookies = browser.find_element_by_id("onetrust-accept-btn-handler")
cookies.click()

names = []

soup = BeautifulSoup(browser.page_source, "html.parser")
name = soup.find_all("td", {"class": "first"})

for n in name:
    names.append(n.find("a").get_text())

print(names)

browser.quit()

C:\Users\Arthur\AppData\Local\Temp/ipykernel_17640/3970210774.py:10: DeprecationWarning: find_element_by_* commands are deprecated. Please use find_element() instead
  cookies = browser.find_element_by_id("onetrust-accept-btn-handler")


['Phoenix Suns', 'Golden State Warriors', 'Brooklyn Nets', 'Utah Jazz', 'Chicago Bulls', 'Cleveland Cavaliers', 'Memphis Grizzlies', 'Milwaukee Bucks', 'Miami Heat', 'LA Clippers', 'Denver Nuggets', 'Los Angeles Lakers', 'Washington Wizards', 'Boston Celtics', 'Charlotte Hornets', 'Minnesota Timberwolves', 'Philadelphia 76ers', 'Atlanta Hawks', 'Dallas Mavericks', 'Toronto Raptors', 'New York Knicks', 'Indiana Pacers', 'Portland Trail Blazers', 'Sacramento Kings', 'San Antonio Spurs', 'Houston Rockets', 'New Orleans Pelicans', 'Oklahoma City Thunder', 'Orlando Magic', 'Detroit Pistons', 'Phoenix Suns', 'Golden State Warriors', 'Brooklyn Nets', 'Utah Jazz', 'Chicago Bulls', 'Cleveland Cavaliers', 'Memphis Grizzlies', 'Milwaukee Bucks', 'Miami Heat', 'LA Clippers', 'Denver Nuggets', 'Los Angeles Lakers', 'Washington Wizards', 'Boston Celtics', 'Charlotte Hornets', 'Minnesota Timberwolves', 'Philadelphia 76ers', 'Atlanta Hawks', 'Dallas Mavericks', 'Toronto Raptors', 'New York Knicks', 'I

## Plot

Explaining the choices of figures that your program can plot and the new insights that can be derived

In [ ]:
# Lecture des csv créés par le scrapping
player = open("player.csv")
team = open("team.csv")

### Plot 1 : boxplot...

Il a été choisi d'afficher... car...

### Plot 4 : Scattered de taille / fg ou 3p 

Il a été choisi...

### Plot 5 : Correlation en heat

On vérifie...

## Prediction

Comme les val sont corrélés...

#### On crée une regression du model

In [ ]:
myReg = linear_model.LinearRegression()

X = data[:,0]
X=X.reshape(-1,1)   
Y = data[:,2]

poly =  make_pipeline(PolynomialFeatures(degree=2), Ridge())

poly.fit(X, Y)

Y_pred2 = poly.predict(X)

plt.scatter(X, Y,  color='grey')
plt.scatter(X, Y_pred2, color='green', linewidth=3)

plt.show()

#### On entraine le model

In [ ]:
myReg.fit(X,Y)   

#### On prédit ... avec un ... à 2017

In [ ]:
poly.predict([[2017]])

Drawbacks in your program and issues that can be upgraded, tasks that can be added to it

Difficulties encountered while doing the project